## Building Tools - LangChain handbook

> https://www.pinecone.io/learn/series/langchain/langchain-tools/

---

### Simple Calculator Tool

We will start with a simple custom tool. The tool is a simple calculator that calculates a circle’s circumference based on the circle’s radius.



In [9]:
import os
from math import pi
from typing import Union
from dotenv import load_dotenv

In [30]:
# models and embeddings
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import AzureOpenAI, HuggingFaceHub
from langchain.docstore.document import Document

from langchain.embeddings import AzureOpenAIEmbeddings

# prompts and utils
from langchain.prompts import PromptTemplate, FewShotPromptTemplate

# loaders and splitters
from langchain_community.document_loaders import DirectoryLoader, TextLoader, UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter, MarkdownTextSplitter, MarkdownHeaderTextSplitter


# memory
from langchain.memory import ConversationBufferWindowMemory

# chains
from langchain.chains import LLMChain, ConversationChain, RetrievalQA

# vectorstores
from langchain.vectorstores import FAISS
from langchain.vectorstores.chroma import Chroma

# agents and tools
from langchain.tools import BaseTool
from langchain.agents import initialize_agent

In [18]:
load_dotenv(dotenv_path="/Users/shaunaksen/Documents/personal-projects/Natural-Language-Processing/LLM Concepts/llamaindex_tutorials/knowledge_graphs/.env")

True

In [19]:
AZURE_API_KEY = os.getenv('AZURE_API_KEY')
AZURE_API_BASE = os.getenv('AZURE_API_BASE')
AZURE_API_VERSION = os.getenv('AZURE_API_VERSION')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

In [20]:
def init_models() -> list:
    embeddings = AzureOpenAIEmbeddings(
        deployment="text-embedding-ada-002",
        model="text-embedding-ada-002",
        openai_api_type='azure',
        azure_endpoint=AZURE_API_BASE,
        openai_api_key=AZURE_API_KEY,
        openai_api_version=AZURE_API_VERSION,
        chunk_size=1, max_retries=1e0
    )

    llm_chat_gpt_4 = AzureChatOpenAI(deployment_name='gpt-4-32k',
                          model='gpt-4-32k',
                          openai_api_type='azure',
                          azure_endpoint=AZURE_API_BASE,
                          openai_api_key=AZURE_API_KEY,
                          openai_api_version=AZURE_API_VERSION,
                          max_retries=2,
                          temperature=0,
                          streaming=True
                          )

    llm_gpt_4 = AzureOpenAI(deployment_name='gpt-4-32k',
                            model='gpt-4-32k',
                            openai_api_type='azure',
                            azure_endpoint=AZURE_API_BASE,
                            openai_api_key=AZURE_API_KEY,
                            openai_api_version=AZURE_API_VERSION,
                            max_retries=2,
                            temperature=0,
                            streaming=True
                            )

    llm_text_davinci = AzureOpenAI(deployment_name='text-davinci-003',
                          model='text-davinci-003',
                          openai_api_type='azure',
                          azure_endpoint=AZURE_API_BASE,
                          openai_api_key=AZURE_API_KEY,
                          openai_api_version=AZURE_API_VERSION,
                          max_retries=2,
                          temperature=0)

    hub_llm = HuggingFaceHub(
        repo_id="google/flan-t5-large",
        model_kwargs={"temperature": 0}
    )

    return [embeddings, llm_chat_gpt_4, llm_gpt_4, llm_text_davinci, hub_llm]

In [21]:
embeddings, llm_chat_gpt_4, llm_gpt_4, llm_text_davinci, hub_llm = init_models()

In [25]:
text = "Tell me an astronaut joke"
print(llm_text_davinci(text))



Q: What did the astronaut say when he stepped on the moon?
A: "One small step for man, one giant leap for mankind!"


In [27]:
class CircumferenceTool(BaseTool):
    name = "Circumference calculator"
    description = "use this tool when you need to calculate a circumference using the radius of a circle"

    def _run(self, radius: Union[int, float]):
        return float(radius)*2.0*pi
    
    def _arun(self, radius: Union[int, float]):
        raise NotImplementedError("This tool does not support async")

Here we initialized our custom CircumferenceTool class using the BaseTool object from LangChain. We can think of the BaseTool as the required template for a LangChain tool.

We have two attributes that LangChain requires to recognize an object as a valid tool. Those are the name and description parameters.

The description is a natural language description of the tool the LLM uses to decide whether it needs to use it. Tool descriptions should be very explicit on what they do, when to use them, and when not to use them.

> In our description, we did not define when not to use the tool. That is because the LLM seemed capable of identifying when this tool is needed. Adding “when not to use it” to the description can help if a tool is overused.

Following this, we have two methods, _run and _arun. When a tool is used, the _run method is called by default. The _arun method is called when a tool is to be used asynchronously. We do not cover async tools in this chapter, so, for now, we initialize it with a NotImplementedError.

In [31]:
# initialize conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    k=5,
    return_messages=True
)

Here we initialize the LLM with a temperature of 0. A low temperature is useful when using tools as it decreases the amount of “randomness” or “creativity” in the generated text of the LLMs, which is ideal for encouraging it to follow strict instructions — as required for tool usage.

In the conversation_memory object, we set k=5 to “remember” the previous five human-AI interactions.

Now we initialize the agent itself. It requires the llm and conversational_memory to be already initialized. It also requires a list of tools to be used. We have one tool, but we still place it into a list.


In [32]:
tools = [CircumferenceTool()]

agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm_chat_gpt_4,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

The agent type of chat-conversation-react-description tells us a few things about this agent, those are:

- chat means the LLM being used is a chat model. Both gpt-4 and gpt-3.5-turbo are chat models as they consume conversation history and produce conversational responses. A model like text-davinci-003 is not a chat model as it is not designed to be used this way.

- conversational means we will be including conversation_memory.

- react refers to the ReAct framework, which enables multi-step reasoning and tool usage by giving the model the ability to “converse with itself”.

- description tells us that the LLM/agent will decide which tool to use based on their descriptions — which we created in the earlier tool definition.


In [33]:
agent("can you calculate the circumference of a circle that has a radius of 7.81mm")



> Entering new AgentExecutor chain...
```json
{
    "action": "Circumference calculator",
    "action_input": "7.81"
}
```
Observation: 49.071677249072565
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The circumference of a circle with a radius of 7.81mm is approximately 49.07mm."
}
```

> Finished chain.


{'input': 'can you calculate the circumference of a circle that has a radius of 7.81mm',
 'chat_history': [],
 'output': 'The circumference of a circle with a radius of 7.81mm is approximately 49.07mm.'}

In [34]:
7.81 * 2 * pi

49.071677249072565

> This works here (maybe because we are using gpt-4) but in the article this did not work, so for the next step we will assume that the ans returned is not perfect

The Final Answer action is what the agent uses when it has decided it has completed its reasoning and action steps and has all the information it needs to answer the user’s query. That means the agent decided not to use the circumference calculator tool.

LLMs are generally bad at math, but that doesn’t stop them from trying to do math. The problem is due to the LLM’s overconfidence in its mathematical ability. To fix this, we must tell the model that it cannot do math. First, let’s see the current prompt being used:



In [41]:
type(agent)

langchain.agents.agent.AgentExecutor

In [42]:
type(agent.agent)

langchain.agents.conversational_chat.base.ConversationalChatAgent

In [49]:
print (agent.agent.llm_chain.prompt.messages[0])

prompt=PromptTemplate(input_variables=[], template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.\n\nOverall, Assistant is

In [50]:
print (agent.agent.llm_chain.prompt.messages[0].prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful system that can help with a wide range of task

We will add a single sentence that tells the model that it is “terrible at math” and should never attempt to do it.

```
Unfortunately, the Assistant is terrible at maths. When provided with math questions, no matter how simple, assistant always refers to its trusty tools and absolutely does NOT try to answer math questions by itself
```

With this added to the original prompt text, we create a new prompt using agent.agent.create_prompt — this will create the correct prompt structure for our agent, including tool descriptions. Then, we update agent.agent.llm_chain.prompt.

In [55]:
sys_msg = """Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Unfortunately, Assistant is terrible at maths. When provided with math questions, no matter how simple, assistant always refers to it's trusty tools and absolutely does NOT try to answer math questions by itself

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.
"""

In [56]:
new_prompt = agent.agent.create_prompt(
    system_message=sys_msg,
    tools=tools
)

In [57]:
type(new_prompt)

langchain_core.prompts.chat.ChatPromptTemplate

In [58]:
agent.agent.llm_chain.prompt = new_prompt

In [59]:
agent("can you calculate the circumference of a circle that has a radius of 7.81mm")



> Entering new AgentExecutor chain...
```json
{
    "action": "Circumference calculator",
    "action_input": "7.81"
}
```
Observation: 49.071677249072565
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The circumference of a circle with a radius of 7.81mm is approximately 49.07mm."
}
```

> Finished chain.


{'input': 'can you calculate the circumference of a circle that has a radius of 7.81mm',
 'chat_history': [],
 'output': 'The circumference of a circle with a radius of 7.81mm is approximately 49.07mm.'}